In [141]:
'''
2. 处理泰坦尼克号数据集于data/目录里
解压缩此压缩包将提供两个CSV文件，train.csv和test.csv，可用pandas.read_csv()加载
最后目标是训练一个可以根据其他列预测Survived（是否存活）列的分类器
Age与Cabin(客舱)列可能有缺失值
'''
import pandas as pd

'''
有关特征处理的说明：
注意到座舱缺失值过多，考虑到座舱一定程度上会关联到生还结果，因此根据gpt建议将其改为是否有座舱信息
而名称列、乘客id列与票据号列主观判断，不与是否生还直接关联，因此剔出此内容
'''

original_data = pd.read_csv('/ml-teach/homework/HW3/data/test_augmented.csv')

def Adult_or_not(X):
    if X['Age'] < 14:
        return 0
    else:
        return 1
def whats_gender(X):
    if X['Sex'] == 'male':
        return 0
    else:
        return 1

# 特征工程 简化与增加特征
# Fare里有一个缺失值...
original_data['Age'] = original_data['Age'].fillna(original_data['Age'].mean()).astype(int)
original_data['Adult?'] = original_data.apply(Adult_or_not, axis=1)
original_data['Sex'] = original_data.apply(whats_gender, axis=1)
original_data['Fare'] = original_data['Fare'].fillna(original_data['Fare'].mean())
data = original_data.drop(columns=['Name','PassengerId','Ticket'])
data['Cabin'] = data['Cabin'].notna().astype(int)

In [143]:
# 特征工程(填充缺失值) -> 测试集划分 -> 特征工程(归一化，独热编码) -> 管道封装 -> 训练与评分
# 二元分类问题，多特征，含非数值特征，含缺失值，线性回归？
from sklearn.model_selection import StratifiedShuffleSplit

# 特征合并
data['StratLabel'] = data['Adult?'].astype(str) + '_' + data['Sex'].astype(str)

# 分层采样与划分
split = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=35)
for train_index, test_index in split.split(data, data['StratLabel']):
    strat_train_set = data.loc[train_index]
    strat_test_set = data.loc[test_index]

train_X = strat_train_set.drop(columns=['StratLabel', 'Survived'])
train_y = strat_train_set['Survived']
test_X = strat_test_set.drop(columns=['StratLabel', 'Survived'])
test_y = strat_test_set['Survived']

In [151]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression

# 管道建立，数据归一化与独热编码的稀疏矩阵
num_attribs = list(train_X.drop('Embarked', axis=1).columns)
cat_attribs = ['Embarked']

data_pipeline = ColumnTransformer([('Scaler', StandardScaler(), num_attribs),
                          ('1hotencoder', OneHotEncoder(), cat_attribs)])
train_X_prepared = data_pipeline.fit_transform(train_X)
test_X_prepared = data_pipeline.transform(test_X)


#模型预测与评分
model = LinearRegression()
model.fit(train_X_prepared, train_y)
pred_y = (model.predict(test_X_prepared) > 0.5).astype(int)
# print("Test 缺失值情况：\n", test_X.isnull().sum())
from sklearn.metrics import r2_score, mean_squared_error

print('R2分数为：',r2_score(test_y, pred_y), '\n'
      '均方误差为:',mean_squared_error(test_y, pred_y))
pred_y

R2分数为： 0.0389016018306636 
均方误差为: 0.23809523809523808


array([0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0])

In [152]:
'''
总体上按照总结的使用手册笔记完成的，我对sklearn函数的具体实现了解受限于学历很难深入，我几乎不懂高数
因此目前仅是作为黑箱函数，能通用sklearn数据库是当前目标,其抽象原理、概念及实现方式都能理解，
不懂得仅是数学函数本身
'''

# 简答题部分
# 1. 精确率是什么，评估分类的性能为什么不能只用精确率
    # 精确率(Accuracy)用于描述一个分类问题下所有预测中预测正确的比例，
    # 可以描述为 TP+FP中TP的占比

# 2. 简述下混淆矩阵是什么
    # 一个区分预测与实际结果的正例与负例的矩阵，可抽象为此类形式
#     	        实际为正例(1)	        实际为负例(0)
# 预测为正例(1)	TP(True Positive)	FP(False Positive)
# 预测为负例(0)	FN(False Negative)	TN(True Negative)

# 3. 简述下各个性能指标的意思： 准确率，召回率，F1分数，假阳性，PR曲线，ROC曲线，AUC分数
    #准确率： 所有预测中正确的比例
    #召回率： 所有预测为正例的样本中有多少被预测对的(于TP+FN中TP的占比)
    #F1分数： 精确率与召回率的调和平均
    #假阳性： 预测为正例但实际为负例？（FP?）
    #PR曲线： 一个用于展示在不同决策阈值下，精确率与召回率之间的权衡曲线
    #ROC曲线： 一个用于可视化预测器性能的曲线，其横轴为假阳性率，纵轴为召回率
    #AUC分数： ROC曲线下的面积，用于衡量分类器整体性能，值越高则表现越好

# 4. 简述下准确率-召回率权衡
    #你无法既要又要的一个典范，准确率代表了识别的准确度，而召回率则代表了判定为真的数量；
    #想要增加识别的准确率就要提升筛选的门槛，相应的召回率便会降低；
    #而提升召回率则代表着放宽筛选门槛，会识别判断更多的正例，但准确度一定有所下降
    #一个模型需要根据实际场景达成一个符合情景的准确率-召回率权衡，像是肿瘤预测这种高危判断必然要提升召回率

# 5. 如何用二元分类器 去解决多元分类的问题
    #一个简单粗暴的方法是利用许多二元分类器，判断内容‘是不是我的目标内容’

# 6. 什么是 多标签-多分类问题？
    #区别于一般的每个数据只有一个标签的类型，这类问题一个数据可能有多个标签，同时也可能被归入多种近似类别
    #举个例子，一部电影的标签可以有多个，而这部电影也可能被归入多种类别中(浪漫恐怖，爱情，奇幻电影)

'\n总体上按照总结的使用手册笔记完成的，我对sklearn函数的具体实现了解受限于学历很难深入，我几乎不懂高数\n因此目前仅是作为黑箱函数，能通用sklearn数据库是当前目标,其抽象原理、概念及实现方式都能理解，\n不懂得仅是数学函数本身\n'